In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import tqdm
import annotation, misc, hyspec_io, image_render, hyspec_ml, hyspec_cnn, preprocess
import skimage.exposure
import tensorflow as tf

In [3]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [4]:
# Paths
json_gray = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_Annotation/20210825_Olbergholmen_Annotation_20230228_gray/label_classes.json')
train_dataset_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/5b_Rad_Georef_SGC_PCA_TrainValSplit/Training_Images')
train_tiles_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/5b_Rad_Georef_SGC_PCA_TrainValSplit/Training_Tiles')
train_spectra_path = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/5b_Rad_Georef_SGC_PCA_TrainValSplit/Training_Spectra/20210825_OlbergAreaS_Spectra.npz')
merged_classes_json_tiles = train_tiles_dir / '20210825_OlbergAreaS_MergedClasses.json'
merged_classes_json_spectra = train_spectra_path.parent / '20210825_OlbergAreaS_MergedClasses.json'


In [5]:
# Parameters
TILE_SHAPE = (128,128)

# Instructions for merging / extracting classes
classes_to_merge = [['Zostera marina - dominant','Zostera marina - normal','Zostera marina - sparse','Zostera marina - NGT'],
                    ['Rockweed','Rockweed with turf algae','Fucus serratus','Saccharina latissima','Rockweed - NGT'],
                    ['Turf algae - dominant','Turf algae - normal','Turf algae - sparse','Aegagropila (algae ball)','Other algae - NGT']]
merged_class_names = ['Zostera marina',
                      'Rockweed',
                      'Other algae']
classes_to_extract = ['Sand',
                      'Zostera marina',
                      'Zostera marina with turf algae',
                      'Rockweed',
                      'Other algae']


In [6]:
# Read annotation metadata file, show classes
class_dict = annotation.read_hasty_metadata(json_gray)

In [7]:
# Loop through PCA images, merge labels and create tiles
tiles_dataset_list = []
for dataset_path in train_dataset_dir.glob('*'):
    dataset = tf.data.experimental.load(str(dataset_path))
    pca_im, label_im = [numpy_data for numpy_data in dataset.as_numpy_iterator()][0]
        
    # Read and filter image (class mask)
    merged_class_dict, merged_class_mask = annotation.merge_classes_with_mask(
        class_dict,label_im,classes_to_merge,merged_class_names)
    filtered_class_dict, filtered_class_mask = annotation.extract_subset(
        merged_class_dict,merged_class_mask,classes_to_extract)
    
    # Split into tiles
    pca_tiles, label_tiles = hyspec_cnn.labeled_image_to_tensor_tiles(pca_im,filtered_class_mask,TILE_SHAPE)

    # Create dataset from tiles
    tile_image_names = tf.convert_to_tensor([dataset_path.name for i in range(len(pca_tiles))])
    tiles_dataset_list.append(tf.data.Dataset.from_tensor_slices((pca_tiles,label_tiles,tile_image_names)))

In [8]:
# Show merged / filtered classes
filtered_class_dict

{'Sand': 1,
 'Zostera marina': 2,
 'Zostera marina with turf algae': 3,
 'Rockweed': 4,
 'Other algae': 5}

In [9]:
# Concatenate tiles
tiles_concat = tiles_dataset_list[0]
for tiles in tiles_dataset_list[1:]:
    tiles_concat = tiles_concat.concatenate(tiles)

In [10]:
print(f'Number of tiles: {tiles_concat.cardinality()}')

Number of tiles: 457


In [11]:
# Save tiles dataset and class dict
tf.data.experimental.save(tiles_concat,str(train_tiles_dir/'Dataset'))
annotation.save_class_dict(filtered_class_dict,merged_classes_json_tiles)

In [12]:
# Extract spectra and classes from dataset
X_list = []
y_list = []
for pca_tile,label_tile,_ in tiles_concat.as_numpy_iterator():
    # nonzero_mask = np.all(pca_tile != 0,axis=2,keepdims=False)
    labelled_mask = (label_tile != 0)
    X_list.append(pca_tile[labelled_mask])
    y_list.append(label_tile[labelled_mask])
X = np.concatenate(X_list)
y = np.concatenate(y_list)

In [13]:
# Save spectra and class disct
np.savez(train_spectra_path,X=X,y=y)
annotation.save_class_dict(filtered_class_dict,merged_classes_json_spectra)

In [14]:
# Test loading spectra
# with np.load(train_spectra_path) as npz_files:
#     X_loaded = npz_files['X']
#     y_loaded = npz_files['y']